# Rapport

### Innledning

I denne rapporten vurderer vi hvordan vi kan bruke Python og Pythons relevante maskinlærings-biblioteker til å lage maskinærings-modeller som predikerer elevers avgangskarakterer.

Vi starter med å lese og tolke dataene. For enkelthets skyld, har vi valgt å bare bruke datasettet med elevenes portugisisk-karakterer, ikke datasettet med elevenes matte-karakterer. Vi valgte datasettet med portugisisk-karakterene fordi dette er størst. Videre forbereder vi dataene, før vi lager tre forskjellige modeller. Forskjellen mellom modellene er hvilke forklaringsvariabler de bruker. Til slutt sammenligner vi de tre modellene og plotter noen av egenskapene deres.

In [5]:
#Importering av relevante Python-bibliotek
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import LeaveOneOut, GridSearchCV, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.inspection import plot_partial_dependence
import xgboost as xgb

#Visningsinstillinger
pd.set_option('display.max_columns', None) 

### Lesing og tolking av dataene

Vi starter med å lese og tolke dataen. Vi ser at vi har 649 rader og 33 kolonner, altså har vi 649 elever og 33 variabler for hver elev. "G3" er avgangskarakteren vi vil predikere, mens de 32 andre variablene er forklaringsvariabler. 

Videre plotter vi variablene... Hvilke variabler og færre.

In [1]:
#Leser og viser dataene
df_tmp = pd.read_csv("student-por.csv")
print(df_tmp.head())
print(f"Antall elever = {df_tmp.shape[0]}")
print(f"Antall variabler = {df_tmp.shape[1]}")

#Plot
#response = np.asarray(df_tmp["G3"])

numeric_plots = ["age", "Medu", "Fedu", "traveltime", "studytime", "failures", "famrel", "goout", "Dalc", "Walc", "health", "absences", "G1", "G2" ]
for variable in numeric_plots:
    df_tmp.plot(kind = "scatter", x = variable, y ="G3")

categorical_plots=["sex", "school", "address", "Pstatus", "Mjob", "Fjob", "guardian", "famsize", "reason", "schoolsup", "famsup", "activities", "paid", "internet", "nursery", "higher", "romantic"]
for variable in categorical_plots:
    df_tmp.boxplot(variable, "G3")

NameError: name 'pd' is not defined

### Forberedelse av dataene


Videre forbereder vi dataene. Vi starter med å sjekke om vi mangler noe data. Det gjør vi ikke. Deretter bruker vi "one hot encoding" på de kategoriske variablene. Vi setter "drop_first = True", for å fjerne unødvendige "dummy"-variabler; vi vil ha en så enkel modell som mulig.  Vi splitter så variablene i forklaringsvariabler X og respons Y. Responsen Y er den samme for modell 1, modell 2 og modell 3, mens forklaringsvariablene varierer for de tre modellene. Vi lager derfor tre lister med forklaringsvariabler: X1, X2 og X3. Til slutt splitter vi X-ene og Y inn i treningssett og testsett. Vi har valgt at treningssettet skal bestå av 20% av det totale datasettet. Dermed består testsettet av 80% av det totale datasettet. 

In [8]:
#Sjekker om det vi mangler noe data
df_tmp.isnull().sum()

#Bruker "one hot encoding" på de kategoriske variablene
categorical_cols=["sex", "school", "address", "Pstatus", "Mjob", "Fjob", "guardian", "famsize", "reason", "schoolsup", "famsup", "activities", "paid", "internet", "nursery", "higher", "romantic"]
df = pd.get_dummies(df_tmp, columns=categorical_cols, prefix=categorical_cols, prefix_sep="_", drop_first = True)

#Splitter forklaringsvariabler X og respons Y
X1 = df.drop(["G3", ], axis=1)
X2 = df[["G1", "G2"]]
X3 = df[["G1", "G2", "failures", "school_MS"]]
Y = df["G3"]

#Splitter treningssett og testsett
X1_train, X1_test, X2_train, X2_test, X3_train, X3_test, Y_train, Y_test = train_test_split(X1, X2, X3, Y, test_size = 0.20)


### Trening og testing

Neste steg er å trene og teste modellene. I treningen og testingen av modellene bruker vi Python-biblioteket XGboost. XGboost lager en skog med boostede regresjonstrær, basert på forskjellige parametere. Vi har valgt å inkludere følgende parametere: eta (læringsrate), max_depth (treets maksdybde), xxx
Vi har valgt akkurat disse parametrene fordi... xxxx Vi bruker XGboost sine innebygde funksjoner for cross-validation til å lage modeller, på treningssettet, med alle mulige kombinasjoner av parametere. Vi velger ut modellen som gir best prediksjon. Hvilken modell som gir best prediksjon, bestemmes av XGboost sin "best_estimator"-funksjon. Til slutt bruker vi modellen på dataene i test-settet til å predikere karakterer, for så å sammenligne de predikerte karakterene med de gitte karakterene i testsettet. For å sammenligne bruker vi gjennomsnittlig kvadrert feil, MSE.

For oversikts skyld, går vi videre gjennom treningen og testingen av én og én modell. 

#### Model 1
Modell 1 bruker alle variablene som forklaringsvariabler. Her ser vi at kombinasjonen av eta = x, max_depth = x, ...xxx gir den beste modellen. Når vi tester denne modellen får vi en MSE på xxx.


#### Model 2
Modell 2 bruker elevenes tidligere karakterer som forklaringsvariabler. Her ser vi at kombinasjonen av eta = x, max_depth = x, ...xxx gir den beste modelln. Når vi teser denne modellen får vi en MSE på xxx.

#### Model 3
Modell 2 bruker elevenes tidligere karakterer, stryk og skoler som forklaringsvariabler. Disse variablene har vi valgt ut, fordi vi anser de som juridisk og etisk rettferdige å bruke. For flere detaljer, se den andre rapporten. Her ser vi at kombinasjonen av eta = x, max_depth = x, ...xxx gir den beste modelln. Når vi teser denne modellen får vi en MSE på xxx.


#### Sammenligning
Når vi sammenligner de tre modellene, ser vi altså at modell x er den beste, y er den nest beste og z er den dårligste. 

In [ ]:
#Setter parametere og modell
params = {"eta": [0.01, 0.02, 0.1, 0.15, 0.2, 0.3], "max_depth": [1, 2, 3, 4, 5], "tree_method": ["auto", "exact", "approx", "hist"], }
xgb_model = xgb.XGBRegressor()
GsCV = GridSearchCV(xgb_model, params, verbose=0, scoring="neg_mean_squared_error", cv=5)

#Model 1
GsCV.fit(X1_train, Y_train)     #Lager modeller med forskjellige kombinasjoner av parametere
model1 = GsCV.best_estimator_   #Den beste modellen
print(GsCV.best_score_)         #Negative MSE for den beste modellen
print(GsCV.best_params_)        #Kombinasjonen av parametere i den beste modellen
prediction1 = model1.predict(X1_test)
print(model1.score(X1_test, Y_test))

#Model 2
GsCV.fit(X2_train, Y_train)      #Lager modeller med forskjellige kombinasjoner av parametere
model2 = GsCV.best_estimator_    #Den beste modellen
print(GsCV.best_score_)          #Negative MSE for den beste modellen
print(GsCV.best_params_)         #Kombinasjonen av parametere i den beste modellen
prediction2 = model2.predict(X2_test)
print(model2.score(X2_test, Y_test))

#Model 3
GsCV.fit(X3_train, Y_train)       #Lager modeller med forskjellige kombinasjoner av parametere
model3 = GsCV.best_estimator_     #Den beste modellen
print(GsCV.best_score_)           #Negative MSE for den beste modellen
print(GsCV.best_params_)          #Kombinasjonen av parametere i den beste modellen
prediction_3 = model3.predict(X3_test)
print(model3.score(X3_test, Y_test))

### Plot

Til slutt vil vi plotte viktigheten av de forskjellige forklaringsvariablene og avhengighetene til modellene. Vi ser samtidig på hvordan det påvirker elevenes karakterer å fjerne eller endre noen av forklaringsvariablene. 

Vi starter med å plotte viktigheten av de forskjellige forklaringsvariablene for de tre modellene.

In [ ]:
#Plotter viktigheten av forklaringsvariablene

#Modell 1
model1_features = model1.feature_importances_
plt.subplot(1,3,1)
plt.hist(model1_features)

#Modell 2
model2_features = model1.feature_importances_
plt.subplot(1,3,2)
plt.hist(model2_features)

#Modell 3
model3_features = model1.feature_importances_
plt.subplot(1,3,3)
bins = np.linspace(0, 2, 40)
plt.hist(model3_features)

Vi plotter så avhengigheten.

In [10]:
#Plotter avhengigheten

#Modell 1
#plot_partial_dependence(model1, X1_test, X1_test.columns.values)

#Modell 2
#plot_partial_dependence(model2, X2_test, X2.columns.values)

#Modell 3
#plot_partial_dependence(model3, X3_test, X3_test.columns.values)

NameError: name 'model2' is not defined